In [ ]:
# 将 llm_toy/src 加入 sys.path（兼容多启动位置）
import sys
from pathlib import Path

def _add_src_path():
    candidates = [
        Path.cwd() / 'llm_toy' / 'src',           # 在项目根启动Jupyter
        Path.cwd() / 'src',                        # 在 llm_toy 目录启动
        Path.cwd().parent / 'llm_toy' / 'src',
        Path.cwd().parent / 'src',
    ]
    # 向上回溯几层尝试
    for base in list(Path.cwd().parents)[:3]:
        candidates.append(base / 'llm_toy' / 'src')
        candidates.append(base / 'src')
    for p in candidates:
        if (p / 'model.py').exists() and (p / 'utils.py').exists():
            sys.path.append(str(p.resolve()))
            print('已添加src路径:', p.resolve())
            return str(p.resolve())
    print('警告：未找到 llm_toy/src，请手动添加路径或调整工作目录。')
    return None

SRC_PATH = _add_src_path()


# 03 训练入门：用小数据微训练 GPT-2 (教学版)

本Notebook聚焦动手实践：在一个极小toy数据集上跑通完整训练流程，理解关键步骤与参数。

说明与注释为中文，专有名词如 GPT-2、Tokenizer、DataLoader、Fine-tuning 等保持英文原名。

提示：首次使用 `transformers` 加载模型可能需要联网下载权重；若无网络，请确保本机已有缓存。


In [ ]:
# 环境与导入
import os
from pathlib import Path
import torch
from torch.utils.data import DataLoader

# 便于直接导入 llm_toy/src（已在开头插入path逻辑）
import sys

from model import SimpleGPTModel
from trainer import LLMTrainer, SimpleDataset
from trainer import create_sample_data

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


## 准备数据

我们混合两类toy数据：
- `create_sample_data()` 生成的英文短句
- `llm_toy/data/tiny_corpus.txt` 的中文段落

目标：快速拼出一个小语料，演示 Tokenizer、Dataset、DataLoader 的基本用法。


In [ ]:
# 读取 toy 语料（解析路径多候选）
from pathlib import Path as _P2
_cwd = _P2.cwd()
_cands = [
    _cwd/'llm_toy'/'data'/'tiny_corpus.txt',
    _cwd/'data'/'tiny_corpus.txt',
    _cwd.parent/'llm_toy'/'data'/'tiny_corpus.txt',
]
for base in list(_cwd.parents)[:3]:
    _cands.append(base/'llm_toy'/'data'/'tiny_corpus.txt')
tiny_path = None
for _p in _cands:
    if _p.exists(): tiny_path = _p; break
if tiny_path is None:
    print('警告：未找到 tiny_corpus.txt，仅使用英文sample数据。')
corpus_text = tiny_path.read_text(encoding='utf-8').strip().split('
') if tiny_path.exists() else []

sample_texts = create_sample_data()
texts = corpus_text + sample_texts
len(texts), texts[:2]


## 初始化模型与Tokenizer

说明：`SimpleGPTModel` 封装了 GPT-2 与 Tokenizer。首次加载可能需要下载权重。


In [ ]:
try:
    simple = SimpleGPTModel(model_name='gpt2')
except Exception as e:
    print('加载模型失败：', e)
    print('若无网络，请确认本地已缓存 gpt2 权重或稍后联网重试。')
    raise

tokenizer = simple.tokenizer
base_model = simple.model
base_model.to(device); device


## 构建Dataset与DataLoader

- `SimpleDataset` 会将文本用 Tokenizer 编码为 `input_ids` 与 `attention_mask`
- `max_length` 控制截断/填充长度；演示用 64 即可
- `batch_size` 用 4，CPU 也可运行


In [ ]:
train_ds = SimpleDataset(texts=texts, tokenizer=tokenizer, max_length=64)
train_dl = DataLoader(train_ds, batch_size=4, shuffle=True)
len(train_ds)


## 训练循环（教学化）

我们使用 `LLMTrainer` 封装：
- Optimizer: AdamW
- Scheduler: Linear Warmup
- 训练 epoch：1（示例足够）

练习：尝试修改 `learning_rate` 与 `num_epochs`，观察loss变化。


In [ ]:
trainer = LLMTrainer(model=base_model, tokenizer=tokenizer, device=str(device))
history = trainer.train(
    train_dataloader=train_dl,
    val_dataloader=None,
    num_epochs=1,
    learning_rate=5e-5,
    warmup_steps=0,
    # 保存路径（默认放到项目下 llm_toy/checkpoints）
    save_dir=str((_cwd/'llm_toy'/'checkpoints'))
)
history[:2]


## 简单生成（Sanity Check）

使用训练后的权重进行短文本生成。注意：少量步骤训练不会带来显著能力提升，重在跑通流程。


In [ ]:
prompt = '自然语言处理的核心是'
inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)
with torch.no_grad():
    out_ids = base_model.generate(
        inputs, max_length=50, do_sample=True, temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )
print(tokenizer.decode(out_ids[0], skip_special_tokens=True))


## 练习建议

- 改变 `batch_size`、`max_length`、`temperature` 并记录现象
- 替换语料，尝试更偏向某个领域的文本
- 增加 `num_epochs` 看loss下降趋势（CPU会慢，建议少量步数）
